In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
import pandas as pd

In [3]:
# df = pd.read_csv('WELFake_Dataset.csv')
# df.head()
# df.tail()

In [4]:
# Creating a Spark Session
spark = SparkSession.builder.appName("FakeNewsDetection").getOrCreate()

23/12/10 20:23:58 WARN Utils: Your hostname, Kamals-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.149 instead (on interface en0)
23/12/10 20:23:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/10 20:23:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Load the dataset into Spark DataFrame
dataset = 'WELFake_Dataset.csv'
schema = StructType([
    StructField('Index',IntegerType(),True),
    StructField('title',StringType(),True),
    StructField('text',StringType(),True),
    StructField('label',IntegerType(),True)
])
df = spark.read.format('csv').option('header','true').option("mode", "DROPMALFORMED").schema(schema).load(dataset)
df = df.na.drop(subset=["text"])
df = df.na.drop(subset=["label"])
df.count()

58634

In [6]:
# Tokenize the text data
tokenizer = Tokenizer(inputCol='text',outputCol='tokenized_text')
# df = tokenizer.transform(df)

In [7]:
# Converting words to features using HashingTF
hashingTF = HashingTF(inputCol='tokenized_text',outputCol='features')
# df = hashingTF.transform(df)

In [8]:
# split the data into training and testing sets
train, test = df.randomSplit([0.8,0.2], seed=42)

In [9]:
# create a logistic regression model
lr = LogisticRegression(featuresCol='features',labelCol='label')

In [10]:
# Create a pipleine with the stages
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [11]:
# Train the model
model = pipeline.fit(train)

23/12/10 20:24:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , title, text, label
 Schema: Index, title, text, label
Expected: Index but found: 
CSV file: file:///Users/kamalnath/Documents/BDAT/1008_Data_Collection_Curation/Final%20Project/WELFake_Dataset.csv
23/12/10 20:24:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/10 20:24:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , title, text, label
 Schema: Index, title, text, label
Expected: Index but found: 
CSV file: file:///Users/kamalnath/Documents/BDAT/1008_Data_Collection_Curation/Final%20Project/WELFake_Dataset.csv


In [12]:
predictions = model.transform(test)

In [13]:
# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol='label',rawPredictionCol='rawPrediction',metricName='areaUnderROC')
area_under_roc = evaluator.evaluate(predictions)
print('Area under ROC: {:.2f}'.format(area_under_roc))

23/12/10 20:24:43 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
23/12/10 20:24:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , title, text, label
 Schema: Index, title, text, label
Expected: Index but found: 
CSV file: file:///Users/kamalnath/Documents/BDAT/1008_Data_Collection_Curation/Final%20Project/WELFake_Dataset.csv


Area under ROC: 0.99


In [14]:
model.save('fake_news_model')

23/12/10 20:24:51 WARN TaskSetManager: Stage 220 contains a task of very large size (2097 KiB). The maximum recommended task size is 1000 KiB.


In [122]:
spark.stop()